<a href="https://colab.research.google.com/github/Next-Sunshine/TFF0318/blob/master/TestForTFF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#环境测试
#@test {"skip":true}
!pip install --quiet --upgrade tensorflow_federated
%load_ext tensorboard

     |████████████████████████████████| 430kB 3.5MB/s 
     |████████████████████████████████| 296kB 43.0MB/s 
     |████████████████████████████████| 2.2MB 35.8MB/s 
     |████████████████████████████████| 2.8MB 36.5MB/s 
     |████████████████████████████████| 421.8MB 30kB/s 
     |████████████████████████████████| 20.0MB 249kB/s 
     |████████████████████████████████| 102kB 11.1MB/s 
     |████████████████████████████████| 3.9MB 30.1MB/s 
     |████████████████████████████████| 450kB 35.8MB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.


In [3]:
import collections

import numpy as np
import random
import tensorflow as tf
import tensorflow_federated as tff

tf.compat.v1.enable_v2_behavior()

np.random.seed(0)

tff.federated_computation(lambda: 'Hello, world!')()

b'Hello, world!'

In [4]:
#装载数据集，实验使用EMNIST数据集
emnist_train, emnist_test = tff.simulation.datasets.emnist.load_data()
class MNISTLoader():
    def __init__(self):
        #emnist = tff.simulation.datasets.emnist
        (self.train_data, self.train_label), (self.test_data, self.test_label) = emnist_train, emnist_test
        # MNIST中的图像默认为uint8（0-255的数字）。以下代码将其归一化到0-1之间的浮点数，并在最后增加一维作为颜色通道
        self.train_data = np.expand_dims(self.train_data.astype(np.float32) / 255.0, axis=-1)      # [60000, 28, 28, 1]
        self.test_data = np.expand_dims(self.test_data.astype(np.float32) / 255.0, axis=-1)        # [10000, 28, 28, 1]
        self.train_label = self.train_label.astype(np.int32)    # [60000]
        self.test_label = self.test_label.astype(np.int32)      # [10000]
        self.num_train_data, self.num_test_data = self.train_data.shape[0], self.test_data.shape[0]
 
    def get_batch(self, batch_size):
        # 从数据集中随机取出batch_size个元素并返回
        index = np.random.randint(0, np.shape(self.train_data)[0], batch_size)
        return self.train_data[index, :], self.train_label[index]

97402880/97398400 [==============================] - 2s 0us/step


In [0]:
#此处得到客户端的总数K，因为每一轮随机选择C×K的向上取整
#根据FedAvg做的研究C取0.2可获得较好的性能，为了简化模型K这里取固定的数
K = len(emnist_train.client_ids)
C = 0.2

In [0]:
#NUM_CLIENTS在实验的时候应该是K×C
NUM_CLIENTS = 10
NUM_EPOCHS = 5
BATCH_SIZE = 20
SHUFFLE_BUFFER = 100
PREFETCH_BUFFER = 10

#预处理，将图片中的‘pixels'、‘label'分别表示成x和y
#将28×28的图像展平成784个元素，打乱顺序
def preprocess(dataset):
  #内部函数将像素和标签转换成x和y，并将像素展平
  def batch_format_fn(element):
    return collections.OrderedDict(
        x = tf.reshape(element['pixels'], [-1,28,28,1]),
        #x = np.expand_dims(element['pixels'], axis=-1),
        #x = element['pixels'],
        y = tf.reshape(element['label'], [-1,1])
    )
  
  return dataset.repeat(NUM_EPOCHS).shuffle(SHUFFLE_BUFFER).batch(BATCH_SIZE).map(batch_format_fn).prefetch(PREFETCH_BUFFER)

In [0]:
#为指定用户创建联邦数据,接收训练集和用户id
def make_federated_data(client_data, client_ids):
  return [
      preprocess(client_data.create_tf_dataset_for_client(x))
      for x in client_ids
    ]


In [0]:
#这里改成随机获得NUM_CLIENTS个用户，模拟联邦学习每轮的随机选择用户
 sample_clients = random.sample(emnist_train.client_ids, NUM_CLIENTS)
# sample_clients #说明随机用户的id是取出来了的
# type(emnist_train.client_ids)  #client_ids是一个列表

federated_train_data = make_federated_data(emnist_train, sample_clients)

间隔界

In [0]:
#！自己添加代码中，正在想办法把训练模型改成CNN
class CNN(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.conv1 = tf.keras.layers.Conv2D(
            filters=32,             # 卷积层神经元（卷积核）数目
            kernel_size=[5, 5],     # 感受野大小
            padding='same',         # padding策略（vaild 或 same）
            activation=tf.nn.relu   # 激活函数
        )
        self.pool1 = tf.keras.layers.MaxPool2D(pool_size=[2, 2], strides=2)
        self.conv2 = tf.keras.layers.Conv2D(
            filters=64,
            kernel_size=[5, 5],
            padding='same',
            activation=tf.nn.relu
        )
        #print("kernel:",kernel_size)
        self.pool2 = tf.keras.layers.MaxPool2D(pool_size=[2, 2], strides=2)
        self.flatten = tf.keras.layers.Reshape(target_shape=(7 * 7 * 64,))
        self.dense1 = tf.keras.layers.Dense(units=1024, activation=tf.nn.relu)
        self.dense2 = tf.keras.layers.Dense(units=10)
 
    def call(self, inputs):
        x = self.conv1(inputs)                  # [batch_size, 28, 28, 32]
        x = self.pool1(x)                       # [batch_size, 14, 14, 32]
        x = self.conv2(x)                       # [batch_size, 14, 14, 64]
        x = self.pool2(x)                       # [batch_size, 7, 7, 64]
        x = self.flatten(x)                     # [batch_size, 7 * 7 * 64]
        x = self.dense1(x)                      # [batch_size, 1024]
        x = self.dense2(x)
        #tf.nn.softmax()将原始输出归一化，且能凸显原始向量中最大的值                      # [batch_size, 10]
        output = tf.nn.softmax(x)
        return output

In [0]:
#！此处也是自己添加代码
num_epochs = 5
batch_size = 50
learning_rate = 0.001

#实例化模型
model = CNN()
#data_loader = MNISTLoader()
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

间隔界


In [18]:
#创建一个变量集合来表示所有变量,包括model(weights+bias)以及metrics(num_examples,loss_sum,accuracy_sum)
# MnistVariables = collections.namedtuple(
#     'MnistVariables','weights bias num_examples loss_sum accuracy_sum'
# )
MnistVariables = collections.namedtuple(
    'MnistVariables','conv_kernel pool_kernel num_examples loss_sum accuracy_sum'
)

ERROR! Session/line number was not unique in database. History logging moved to new session 59


In [0]:
#创建变量并初始化
def create_mnist_variables():
  return MnistVariables(
      # weights=tf.Variable(
      #     # lambda: tf.zeros(dtype=tf.float32, shape=(784,10)),
      #     lambda: tf.zeros(dtype=tf.float32, shape=(28,28,10)),
      #     name='weights',
      #     trainable=True),
      # bias=tf.Variable(
      #     lambda: tf.zeros(dtype=tf.float32, shape=(10)),
      #     name='bias',
      #     trainable=True),
      kernel=tf.Variable(
          # lambda: tf.zeros(dtype=tf.float32, shape=(784,10)),
          lambda: tf.zeros(dtype=tf.float32, shape=(5,5)),
          name='conv_kernel',
          trainable=True),
      kernel=tf.Variable(
          # lambda: tf.zeros(dtype=tf.float32, shape=(784,10)),
          lambda: tf.zeros(dtype=tf.float32, shape=(2,2)),
          name='pool_kernel',
          trainable=True),
      num_examples=tf.Variable(0.0, name='num_examples', trainable=False),
      loss_sum=tf.Variable(0.0, name='loss_sum', trainable=False),
      accuracy_sum=tf.Variable(0.0, name='accuracy_sum', trainable=False)
  )

In [0]:
#自定义前向传播函数
def mnist_forward_pass(variables, batch):
  X = batch['x']  #取训练图片以及
  print(str(X)) #形状是[None, 784]，为什么预处理没有生效
  y = batch['y'] #取标签
  with tf.GradientTape() as tape:
    y_pred = model(X)
    loss = tf.keras.losses.sparse_categorical_crossentropy(y_true=y, y_pred=y_pred)
    loss = tf.reduce_mean(loss)
    #print("loss %f" % loss)
  grads = tape.gradient(loss, model.variables)
  optimizer.apply_gradients(grads_and_vars=zip(grads, model.variables))
  predictions = tf.cast(tf.argmax(y_pred,1), tf.int32)
  '''
  #！后面为原版代码
  y = tf.nn.softmax(tf.matmul(batch['x'], variables.weights) + variables.bias)
  predictions = tf.cast(tf.argmax(y,1), tf.int32) 

  flat_labels = tf.reshape(batch['y'], [-1])
  #计算交叉熵损失
  loss = -tf.reduce_mean(
      tf.reduce_sum(tf.one_hot(flat_labels, 10) * tf.math.log(y), axis=[1]))
  '''
  #计算准确率
  flat_labels = tf.reshape(batch['y'], [-1])
  accuracy = tf.reduce_mean(
      tf.cast(tf.equal(predictions, flat_labels), tf.float32))
  
  #样本数
  num_examples = tf.cast(tf.size(batch['y']), tf.float32)

  #更新样本数、损失和、精度和,每一批都考虑了自己的权重
  variables.num_examples.assign_add(num_examples)
  variables.loss_sum.assign_add(loss * num_examples)
  variables.accuracy_sum.assign_add(accuracy * num_examples)

  return loss, predictions

In [0]:
#计算本地用户的metrics度量
def get_local_mnist_metrics(variables):
  return collections.OrderedDict(
      num_examples=variables.num_examples,
      loss=variables.loss_sum / variables.num_examples,
      accuracy=variables.accuracy_sum / variables.num_examples
  )

In [0]:
#集合每个设备发出的本地度量
@tff.federated_computation
def aggregate_mnist_metrics_across_clients(metrics):
  return collections.OrderedDict(
      num_examples=tff.federated_sum(metrics.num_examples),
      loss=tff.federated_mean(metrics.loss, metrics.num_examples),
      accuracy=tff.federated_mean(metrics.accuracy, metrics.num_examples)
  )

In [0]:
#自定义模型，创建tff.learning.model实例
class MnistModel(tff.learning.Model):

  def __init__(self):
    self._variables = create_mnist_variables()

  #所有的“tf.Variables”都应该在“__init__”中引入
  @property
  def trainable_variables(self):
    #return [self._variables.weights, self._variables.bias]
    return [self._variables.conv_kernel, self._variables.pool_kernel]
  
  @property
  def non_trainable_variables(self):
    return []
  
  @property
  def local_variables(self):
    return [self._variables.num_examples, self._variables.loss_sum,
         self._variables.accuracy_sum]
  
  @property
  def input_spec(self):
    return collections.OrderedDict(
        # x=tf.TensorSpec([None, 784], tf.float32),
        x=tf.TensorSpec([None, 28,28,1], tf.float32),
        y=tf.TensorSpec([None, 1], tf.int32)
    )

  @tf.function
  def forward_pass(self, batch, training=True):
    del training
    loss, predictions = mnist_forward_pass(self._variables, batch)
    #取出样本数
    num_examples = tf.shape(batch['x'])[0]
    return tff.learning.BatchOutput(
        loss=loss, predictions=predictions, num_examples=num_examples)
    
  @tf.function
  def report_local_outputs(self):
    return get_local_mnist_metrics(self._variables)
  
  @property
  def federated_output_computation(self):
    return aggregate_mnist_metrics_across_clients

In [29]:
#创建迭代器执行联合平均的迭代过程
iterative_process = tff.learning.build_federated_averaging_process(
    MnistModel,
    client_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=0.02)
)

Tensor("batch:0", shape=(None, 28, 28, 1), dtype=float32)


ValueError: ignored

In [0]:
#获得初始状态
state = iterative_process.initialize()

In [21]:
#查看第一轮训练
sample_clients = random.sample(emnist_train.client_ids, NUM_CLIENTS)
federated_train_data = make_federated_data(emnist_train, sample_clients)
state, metrics = iterative_process.next(state, federated_train_data)
print('round 1, metrics={}'.format(metrics))

round 1, metrics=<num_examples=5270.0,loss=2.3192951679229736,accuracy=0.1318785548210144>


In [22]:
#计算更多轮
NUM_ROUNDS = 11
for round_num in range(2,NUM_ROUNDS):
  #随机选择NUM_CLIENTS个用户
  sample_clients = random.sample(emnist_train.client_ids, NUM_CLIENTS)
  federated_train_data = make_federated_data(emnist_train, sample_clients)

  state, metrics = iterative_process.next(state, federated_train_data)
  print('round {:2d}, metrics={}'.format(round_num, metrics))

round  2, metrics=<num_examples=5055.0,loss=2.2916009426116943,accuracy=0.1705242395401001>
round  3, metrics=<num_examples=5215.0,loss=2.298238515853882,accuracy=0.14439117908477783>
round  4, metrics=<num_examples=4450.0,loss=2.3351666927337646,accuracy=0.1323595494031906>
round  5, metrics=<num_examples=5080.0,loss=2.2978105545043945,accuracy=0.15216535329818726>
round  6, metrics=<num_examples=4860.0,loss=2.3288588523864746,accuracy=0.13724279403686523>
round  7, metrics=<num_examples=5325.0,loss=2.305818557739258,accuracy=0.13971830904483795>
round  8, metrics=<num_examples=5100.0,loss=2.306328535079956,accuracy=0.15529412031173706>
round  9, metrics=<num_examples=5065.0,loss=2.323202133178711,accuracy=0.13780848681926727>
round 10, metrics=<num_examples=5045.0,loss=2.2971935272216797,accuracy=0.16233894228935242>


In [0]:
'''
#使用tensorboard可视化
#使用Tensorboard可视化这些联邦计算的度量
#创建目录和相应的摘要编写器
#@test {"skip": true}
logdir = "/tmp/logs/scalars/training/"
summary_writer = tf.summary.create_file_writer(logdir)
state = iterative_process.initialize()

#!!!太坑了，我以为@test是无关紧要的东西，emmm，就省略了，没想到就凉凉
#@test {"skip": true}
with summary_writer.as_default():
  for round_num in range(1, NUM_ROUNDS):
    #随机选择用户
    sample_clients = random.sample(emnist_train.client_ids, NUM_CLIENTS)
    federated_train_data = make_federated_data(emnist_train, sample_clients)

    state, metrics = iterative_process.next(state, federated_train_data)

    print('round {:2d}, metrics={}'.format(round_num, metrics))

    for name, value in metrics._asdict().items():
      tf.summary.scalar(name, value, step=round_num)

#这个tensorBoard像有猫病一样，昨天还可以今天来突然就不行了
#@test {"skip":true}
#%load_ext tensorboard
%tensorboard --logdir /tmp/logs/scalars/ --port=0
'''

round  1, metrics=<num_examples=5130.0,loss=3.049107074737549,accuracy=0.12222222238779068>
round  2, metrics=<num_examples=4090.0,loss=2.928187370300293,accuracy=0.15843521058559418>
round  3, metrics=<num_examples=5200.0,loss=2.9345836639404297,accuracy=0.14615385234355927>
round  4, metrics=<num_examples=5140.0,loss=2.914067029953003,accuracy=0.14922179281711578>
round  5, metrics=<num_examples=5080.0,loss=2.794748067855835,accuracy=0.1698818951845169>
round  6, metrics=<num_examples=4775.0,loss=2.7447190284729004,accuracy=0.1736125648021698>
round  7, metrics=<num_examples=5215.0,loss=2.490943193435669,accuracy=0.22205176949501038>
round  8, metrics=<num_examples=5325.0,loss=2.353219985961914,accuracy=0.23868544399738312>
round  9, metrics=<num_examples=4825.0,loss=2.181143045425415,accuracy=0.2878756523132324>
round 10, metrics=<num_examples=5200.0,loss=2.17901873588562,accuracy=0.29750001430511475>


ERROR: Failed to launch TensorBoard (exited with 1).
Contents of stderr:
2020-04-01 01:27:33.614596: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer.so.6'; dlerror: libnvinfer.so.6: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2020-04-01 01:27:33.614725: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer_plugin.so.6'; dlerror: libnvinfer_plugin.so.6: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2020-04-01 01:27:33.614739: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:30] Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
Traceback (most recent call last):
  File "/usr/local/bin/tensorboard", line 8, 